<a href="https://colab.research.google.com/github/almiab1/MUIIADeepLearning/blob/main/PracticaFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica Final | Deep Learning

En el presente trabajo se proponen dos modelos basados en redes convolucionales, los cuales se trata de clasificar correctamente si un individuo está parpadeando o no. En la actualidad se pueden encontrar modelos de gran calidad, los cuales permiten construir modelos de gran precisión basándose sobre la base de dichos modelos pre-entrenados. Por ello, los modelos propuestos en este trabajo se definen haciendo uso de modelos previamente entrenados.

En primer lugar, se propone una red basada en el modelo `InceptionResNetV2`. Un modelo pesado de gran profundidad que proporciona buenos resultados en problemas de clasificación. Se propone una estrategia donde el modelo `InceptionResNetV2` toma la función de extractor de características, obteniendo así las características profundas que posteriormente podrán ser utilizadas en la clasificación de las imágenes. En segundo punto, se propone una red basada en el modelo `DenseNet`. Un modelo de menor tamaño y que permite obtener resultados similares a los de `InceptionResNetV2`. En este caso se trata de re-entrenar el clasificador y aplicando `fine tuning` con el fin de ajustar los parámetros del modelo.

Dado que se parte de un conjunto de datos desbalanceado, es necesario aplicar algunos métodos de balanceo con el fin de obtener un modelo más robusto. Previamente, al entrenamiento de la red, se propone el uso de técnicas como la estratificación de datos o `data augmentation`. A su vez, para mejorar el proceso de entrenamiento se hace empleo de `mini-batches` y de optimizadores como el optimizador Adam.

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
from sklearn.model_selection import train_test_split

from tensorflow.keras import applications, optimizers, Input

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Concatenate
from keras.optimizers import Adam
from keras.applications import InceptionResNetV2

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive

# Montamos el Google Drive en el directorio del proyecto y descomprimios el fichero con los datos
drive.mount('/content/drive')
!unzip -n '/content/drive/MyDrive/UIMP/Asignaturas/5-DeepLearning/Practicas/Final/Source/RT-BENE.zip' >> /dev/null

Mounted at /content/drive


## Preparar datos de entrenamiento

*   **Cargar el dataset en el colab**

In [ ]:
# Especificamos los paths al directorio que contiene las imagenes y al fichero con las etiquetas
data_path = 'RT-BENE/'
imgs_path = data_path + "images/"
labels_path = data_path + "blinks.csv"

# Leemos el fichero CSV con las etiquetas
labels = pd.read_csv(labels_path, dtype = {"class": "category"})

# Supongamos que tienes un DataFrame llamado 'labels' con los nombres de las imágenes y las etiquetas 'blink'
# Convertir la columna 'blink' a tipo string
labels['blink'] = labels['blink'].astype(str)

# Mostramos los primero elementos del dataset
labels.head()

,blink_id,left_eye,right_eye,video,blink
0,0,0_left_000001_rgb.png,0_right_000001_rgb.png,0,0
1,1,0_left_000002_rgb.png,0_right_000002_rgb.png,0,0
2,2,0_left_000003_rgb.png,0_right_000003_rgb.png,0,0
3,3,0_left_000004_rgb.png,0_right_000004_rgb.png,0,0
4,4,0_left_000005_rgb.png,0_right_000005_rgb.png,0,0


- **Creamos las tres particiones de datos: entrenamiento, validación y test**



In [ ]:
# Semilla para replicar los experimentos
seed = 2023
# Creamos las tres particiones de datos: entrenamiento, validación y test
train_data, test_data = train_test_split(labels, test_size=0.2, random_state=seed)
dev_data, test_data = train_test_split(test_data, test_size=0.5, random_state=seed)

En Keras no existen generadores por defecto que devuelvan dos imagenes. Necesitamos crear nuestro propio generador que devuelva a la vez las imágenes de ambos ojos y la etiqueta del frame. Para ello podemos crear dos generadores (uno para cada ojo) usando el método `flow_from_databrame` y combinarlos para crear el generador deseado.

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

# Columnas dataset
left_eye_col = 'left_eye'
right_eye_col = 'right_eye'
y_col = 'blink'

# Parámetros
batch_size = 128
img_width = 122 # minimo 75x75
img_height = 75

# Generador custom que devuelve las dos imagenes de ojos y el label del parpadeo
def generator(dataframe):
  left_eye_generator = datagen.flow_from_dataframe(dataframe=dataframe, 
                                                    directory = imgs_path, 
                                                    target_size =(img_width, img_height), 
                                                    x_col=left_eye_col, 
                                                    y_col=y_col, 
                                                    class_mode="binary", 
                                                    seed=seed, 
                                                    batch_size=batch_size)
  
  right_eye_generator = datagen.flow_from_dataframe(dataframe=dataframe,
                                                    directory = imgs_path,
                                                    target_size =(img_width, img_height),
                                                    x_col=right_eye_col,
                                                    y_col=y_col,
                                                    class_mode="binary",
                                                    seed=seed,
                                                    batch_size=batch_size)
  
  while True:
    left_eye = left_eye_generator.next()
    left_eye_image = left_eye[0]
    label = left_eye[1]
    right_eye = right_eye_generator.next()
    right_eye_image = right_eye[0]
    yield [left_eye_image, right_eye_image], label

# Llamada a la función generator
train_generator = generator(train_data)
dev_generator = generator(train_data)
test_generator = generator(train_data)

In [ ]:
input_shape = (img_width,img_height,3)

# Declaramos dos capas de Input
input_image1 = Input(shape=input_shape)
input_image2 = Input(shape=input_shape)

In [ ]:
# Cargar InceptionResNetV2 pre-entrenado en ImageNet, sin la capa final
base_model = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')

# Congelar todas las capas del modelo base para aplicar transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Extraer las características utilizando InceptionResNetV2
features_image1 = base_model(input_image1)
features_image2 = base_model(input_image2)

# Combinar las características de ambas imágenes
merged_features = Concatenate()([features_image1, features_image2])

# Añadir capas adicionales para la clasificación
x = Dense(1024, activation='relu')(merged_features)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# Crear el modelo
model = Model(inputs=[input_image1, input_image2], outputs=output)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Imprimir un resumen del modelo
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 122, 75, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 input_6 (InputLayer)           [(None, 122, 75, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 inception_resnet_v2 (Functiona  (None, 1536)        54336736    ['input_5[0][0]',                
 l)                                                               'input_6[0][0]']          

In [ ]:
# Entrenamos el modelo con los datos preparados en el punto 2
model.fit(train_generator,
          epochs=10,  # numero de epochs
          verbose=2,  # muestra informacion del error al finalizar cada epoch
          steps_per_epoch=len(train_data)/batch_size,
          validation_data=train_generator,
          validation_steps=len(dev_data)/batch_size)

# Por ultimo, podemos evaluar el modelo en el conjunto de test
print()
test_loss, test_acc = model.evaluate(test_generator,
                                     steps=len(test_data)/batch_size,
                                     verbose=1)
print("test_loss: %.4f, test_acc: %.4f" % (test_loss, test_acc))

Epoch 1/10
670/670 - 3955s - loss: 0.0363 - accuracy: 0.9878 - val_loss: 0.0264 - val_accuracy: 0.9900 - 3955s/epoch - 6s/step
Epoch 2/10


In [ ]:
# Cargamos el modelo InceptionV3 pre-entrenado con ImageNet
base_model = applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))

# Se va a ajustar los parámetros de las nuevas capas del modelo,
# dejando fijos los parámetros del resto de capa (es decir, que no se actualicen durante el entrenamiento)
for layer in base_model.layers:
    layer.trainable = False   # por defecto, el valor de trainable es True

# Añadimos nuevas capas al final para adaptar el modelo a nuestro problema
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

# Añadimos una última capa completamente conectada con 2 neuronas (número de clases) para obtener la salida de la red
predictions = Dense(2, activation='softmax')(x) 

# Creamos el modelo final y lo compilamos
model = Model(inputs=[input_1, input_2], outputs=[predictions])
model.summary()   # representación en modo texto del modelo

model.compile(loss='categorical_crossentropy',                # función de pérdida para problemas de clasificación multi-clase
              optimizer=optimizers.Adam(learning_rate=1e-5),  # optimizador Adam
              metrics=['accuracy'])

# Entrenamos el modelo con los datos preparados en el punto 2
model.fit(train_generator,
          epochs=10,  # numero de epochs
          verbose=2,  # muestra informacion del error al finalizar cada epoch
          steps_per_epoch=len(train_data)/batch_size,
          validation_data=train_generator,
          validation_steps=len(dev_data)/batch_size)

# Por ultimo, podemos evaluar el modelo en el conjunto de test
print()
test_loss, test_acc = model.evaluate(test_generator,
                                     steps=len(test_data)/batch_size,
                                     verbose=1)
print("test_loss: %.4f, test_acc: %.4f" % (test_loss, test_acc))

ValueError: ignored